In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("First President of India was ...")
print(response.content)

Dr. Rajendra Prasad.


In [7]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-43032?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Senior Technical Product Manager

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySenior Technical Product ManagerBeaverton, OregonBecome a Part of the NIKE, Inc. Tea

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Technical Product Manager',
 'experience': '2 years of experience in a computer-related occupation',
 'skills': ['SAP Functional expertise',
  'Domain expertise',
  'Testing knowledge',
  'Design and Dev knowledge',
  'Incident triaging and tracking',
  'Knowledge on testing tool',
  'Purchase business process',
  'Supply chain management processes',
  'Project management',
  'Business process analysis'],
 'description': 'Design and implement data products and features in collaboration with product owners, data analysts, and business partners using Agile / Scrum methodology. Contribute to overall architecture, frameworks and patterns for processing and storing large data volumes. Evaluate and utilize new technologies/tools/frameworks centered around high-volume data processing. Translate product backlog items into engineering designs and logical units of work. Analyze data for the purpose of designing scalable solutions. Define and apply appropriate data acquisition an

In [10]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [11]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [13]:
links = collection.query(query_texts=['Expert in Python',"Expert in Reat Native"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [14]:
job = json_res

In [15]:
job['skills']

['SAP Functional expertise',
 'Domain expertise',
 'Testing knowledge',
 'Design and Dev knowledge',
 'Incident triaging and tracking',
 'Knowledge on testing tool',
 'Purchase business process',
 'Supply chain management processes',
 'Project management',
 'Business process analysis']

In [16]:
collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])

[[{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],